In [21]:
# pixabay에서 구슬 검색 후 모든 페이지의 이미지 다운로드
# 다운로드 폴더 : 검색어
# 파일명 : 페이지_숫자.원래 확장자
# 경로 : 현재 작업 중인 폴더

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from time import sleep
from bs4 import BeautifulSoup
import os
import requests

# 브라우저 최대화
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('headless')
chrome_options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36')
chrome_options.add_argument('lang=ko')
chrome_options.add_argument('--window-size=1960,1080')

item_search = '청개구리'
imgs = {}

driver = webdriver.Chrome(options=chrome_options)

# 1. pixabay 접속
url = 'https://pixabay.com/ko/'
driver.get(url)

sleep(2)

# 2. 검색어 입력
search_bar = driver.find_element(By.CSS_SELECTOR, 'form > input')
search_bar.send_keys(item_search)
search_bar.send_keys(Keys.RETURN)

sleep(1)

# 3. 전체 페이지 추출
pages_text = driver.find_element(By.CSS_SELECTOR, '.paginator > form').text
total_pages = int(pages_text.replace('/', ''))

# 4. 이름이 검색어인 폴더 생성
os.makedirs(item_search, exist_ok=True)

# 폴더 생성 작업이 끝날 시간 딜레이
sleep(2)

# 5. 전체 페이지만큼 반복
for idx in range(1, total_pages+1):
    # 6. a태그 자식 중 alt속성이 있는 이미지 엘리먼트만 추출
    # 마지막 페이지에서 하단에 a태그 자식인 img태그도 추출 
    img_elements = driver.find_elements(By.CSS_SELECTOR, 'a > img[alt]')
    
    # 7. 페이지 내 모든 이미지 추출 후 딕셔너리에 저장
    for num, img in enumerate(img_elements, 1):
        img_ad = img.get_attribute('src')
        
        if img.get_attribute('data-lazy'):
            # 이미지 주소 추출
            img_ad = img.get_attribute('data-lazy')
        else:
            img_ad = img.get_attribute('src')

        # 8. 파일 저장
        # 저장 경로 생성
        save_path = f'{item_search}/{idx}_{num}{img_ad[-4:]}'
        # 이미지 파일 요청
        img_file = requests.get(img_ad).content
        # 이미지 저장
        with open(save_path, 'wb') as file:
            file.write(img_file)

    # 9. 마지막 페이지일 땐 pass, 첫 번째 페이지일 때 첫 번째 a태그 클릭, 나머지는 두 번째 a태그 클릭
    if idx > 1:
        if idx != total_pages:
            driver.find_element(By.XPATH, '//*[@class="paginator"]/a[2]').click()
        else:
            break
    elif idx == 1:
        driver.find_element(By.XPATH, '//*[@class="paginator"]/a').click()

    # 페이지 이동후 딜레이
    sleep(1)
    
driver.close()
print('** 이미지 저장 완료 **')

** 이미지 저장 완료 **
